# Analysis of Public Comments Regarding the EPA's Revised Definition of "Waters of the United States" (WOTUS)

In early 2017, President Trump signed an Executive Order<sup>[1](#footnote1)</sup> requesting that agencies review a 2015 rule regarding the "Waters of the United States". The agencies, including the Environmental Protection Agency (EPA) and the Department of the Army, were instructed to rescind or replace the rule, in accordance with law.

The agencies have since conducted a reevaluation and revision of the definition of "Waters of the United States". Their proposed rule redefines the scope of waters federally regulated under the Clean Water Act (CWA). This rule was open for public comment until April 15, 2019.

This project aims to analyze the content of comments that are publicly available on the [regulations.gov web page](https://www.regulations.gov/docket?D=EPA-HQ-OW-2018-0149) for this docket.

Notes:

<a name="footnote1">1</a>: Executive Order 13778, signed on February 28, 2017, titled "Restoring the Rule of Law, Federalism, and Economic Growth by Reviewing the 'Waters of the United States' Rule"

## Collecting the Data

As of October 2019, there were 625,521 comments received on the docket, of which 11,412 are posted and publicly available under "public submissions". The discrepancy can be attributed to the following note on the site:

>Agencies review all submissions, however some agencies may choose to redact, or withhold, certain submissions (or portions thereof) such as those containing private or proprietary information, inappropriate language, or duplicate/near duplicate examples of a mass-mail campaign.

The regulations.gov site actually has an API to request data programmatically, however API keys are only available to organizations (not individuals). Thus, this project uses a web scraping program to loop over a `.csv` file with a selection of docket information (including unique pages for each public submission) to collect and compile the comments. The original `.csv` file  is saved in the `/Data` directory, or it can be exported directly from the page [to view public comments](https://www.regulations.gov/docketBrowser?rpp=25&so=DESC&sb=commentDueDate&po=0&dct=PS&D=EPA-HQ-OW-2018-0149), the scraper uses a simplified version with comment IDs and URLs only. This file is generated below.

There are two noteworthy items about the web scraper. First, the regulations.gov site is rendered using JavaScript, so the web scraper needs to use the [Selenium package](https://github.com/SeleniumHQ/selenium/) to open a browser, pause to let the page load, then grab the comment. Second, not all comments are available as text on the site, since it allows public submissions in attachment form as well as typing them in directly. Approximately 28% of the comments are in attachment form (most of which are `pdf` format), and the scraper could not collect the information within.

## Library Imports

In [16]:
import glob
import sys
import warnings
warnings.filterwarnings('ignore') 

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [20]:
! echo {sys.prefix}

/Users/heatherkusmierz/anaconda3


In [23]:
# Uncomment the next line to install spaCy
# !conda install --yes --prefix {sys.prefix} spacy

## Sanitize Docket File for Web Scraper Use

This section imports the original docket information file, removes comments in attachment form and saves only the necessary columns (Document ID and Document Detail) for the web scraper.

In [2]:
docket = pd.read_csv('./Data/DOCKET_EPA-HQ-OW-2018-0149.csv', header=2)

docket.head()

,Document Title,Document Type,Attachment Count,Submitter Last Name,Submitter First Name,Organization,Document ID,Posted Date,Received Date,Comment Start Date,Comment Due Date,Implementation Date,Effective Date,Related RINs,Document SubType,Subject,Abstract,Status,Source Citation,OMB Approval Number,FR Citation,Federal Register Number,Start End Page,Special Instructions,Legacy ID,Post Mark Date,File Type,Number Of Pages,Paper Width,Paper Length,Exhibit Type,Exhibit Location,Document Field_1,Document Field_2,Document Detail
0,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,NaN,NaN,NaN,EPA-HQ-OW-2018-0149-6689,04/29/2019,04/11/2019,NaN,NaN,NaN,NaN,NaN,Public Comment,NaN,NaN,Posted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/11/2019,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.regulations.gov/document?D=EPA-HQ-...
1,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,NaN,NaN,NaN,EPA-HQ-OW-2018-0149-6688,04/29/2019,04/11/2019,NaN,NaN,NaN,NaN,NaN,Public Comment,NaN,NaN,Posted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/11/2019,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.regulations.gov/document?D=EPA-HQ-...
2,"Comment submitted by Mike Hook, Craighead Coun...",PUBLIC SUBMISSIONS,1.0,NaN,NaN,NaN,EPA-HQ-OW-2018-0149-6690,04/29/2019,04/11/2019,NaN,NaN,NaN,NaN,NaN,Government Local,NaN,NaN,Posted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/11/2019,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.regulations.gov/document?D=EPA-HQ-...
3,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,NaN,NaN,NaN,EPA-HQ-OW-2018-0149-7377,05/01/2019,04/11/2019,NaN,NaN,NaN,NaN,NaN,Public Comment,NaN,NaN,Posted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/11/2019,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.regulations.gov/document?D=EPA-HQ-...
4,"Comment submitted by Anne Hulick, CT Director,...",PUBLIC SUBMISSIONS,1.0,NaN,NaN,NaN,EPA-HQ-OW-2018-0149-8930,05/04/2019,04/15/2019,NaN,NaN,NaN,NaN,NaN,Company/Organization Comment,NaN,NaN,Posted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04/15/2019,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.regulations.gov/document?D=EPA-HQ-...


In [3]:
docket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11412 entries, 0 to 11411
Data columns (total 35 columns):
Document Title             11412 non-null object
Document Type              11412 non-null object
Attachment Count           3224 non-null float64
Submitter Last Name        0 non-null float64
Submitter First Name       0 non-null float64
Organization               0 non-null float64
Document ID                11411 non-null object
Posted Date                11411 non-null object
Received Date              11411 non-null object
Comment Start Date         0 non-null float64
Comment Due Date           0 non-null float64
Implementation Date        0 non-null float64
Effective Date             0 non-null float64
Related RINs               0 non-null float64
Document SubType           11409 non-null object
Subject                    0 non-null float64
Abstract                   72 non-null object
Status                     11411 non-null object
Source Citation            0 non-null f

In [4]:
# Drop columns with which have all null values
docket.dropna(axis=1, how='all', inplace=True)

# Drop row with withdrawn comment (almost all nulls)
docket.dropna(thresh=6, inplace=True)

docket.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11411 entries, 0 to 11411
Data columns (total 13 columns):
Document Title      11411 non-null object
Document Type       11411 non-null object
Attachment Count    3224 non-null float64
Document ID         11411 non-null object
Posted Date         11411 non-null object
Received Date       11411 non-null object
Document SubType    11409 non-null object
Abstract            72 non-null object
Status              11411 non-null object
Post Mark Date      11353 non-null object
File Type           50 non-null object
Number Of Pages     11411 non-null float64
Document Detail     11411 non-null object
dtypes: float64(2), object(11)
memory usage: 1.2+ MB


In [5]:
# Covert date columns to datetime format
for col in ['Posted Date', 'Received Date']:
    docket[col] = pd.to_datetime(docket[col], format='%m/%d/%Y')

docket.dtypes

Document Title              object
Document Type               object
Attachment Count           float64
Document ID                 object
Posted Date         datetime64[ns]
Received Date       datetime64[ns]
Document SubType            object
Abstract                    object
Status                      object
Post Mark Date              object
File Type                   object
Number Of Pages            float64
Document Detail             object
dtype: object

In [6]:
docket.head()

,Document Title,Document Type,Attachment Count,Document ID,Posted Date,Received Date,Document SubType,Abstract,Status,Post Mark Date,File Type,Number Of Pages,Document Detail
0,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-6689,2019-04-29,2019-04-11,Public Comment,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...
1,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-6688,2019-04-29,2019-04-11,Public Comment,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...
2,"Comment submitted by Mike Hook, Craighead Coun...",PUBLIC SUBMISSIONS,1.0,EPA-HQ-OW-2018-0149-6690,2019-04-29,2019-04-11,Government Local,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...
3,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-7377,2019-05-01,2019-04-11,Public Comment,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...
4,"Comment submitted by Anne Hulick, CT Director,...",PUBLIC SUBMISSIONS,1.0,EPA-HQ-OW-2018-0149-8930,2019-05-04,2019-04-15,Company/Organization Comment,NaN,Posted,04/15/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...


In [7]:
docket['Attachment Count'].value_counts()

1.0     2913
2.0      264
3.0       12
4.0        8
9.0        5
19.0       4
11.0       3
6.0        3
20.0       3
12.0       3
10.0       2
17.0       1
7.0        1
16.0       1
5.0        1
Name: Attachment Count, dtype: int64

In [8]:
docket['File Type'].value_counts()

pdf     35
jpg     10
png      3
docx     2
Name: File Type, dtype: int64

In [12]:
# Simplify DataFrame, save as file for scraper

# Remove comments where it's imbedded in an attachment
scraper = docket[docket['Attachment Count'].isnull()]

# Save only critical info for scraper
scraper = scraper[['Document ID', 'Document Detail']]

print('Total comments posted directly to site: {0:,.0f}'.format(len(scraper)))

# Uncomment next line to generate the simplified scraper CSV file
# scraper.to_csv('./Data/scraper_csv.csv', index=False)

Total comments posted directly to site: 8,187


## Create DataFrame with Comments

This section builds the full DataFrame of docket information plus the text of each comment. It imports all comments into separate DataFrame objects, concatenates them to create one DF of all IDs and comment text, then merges it with the `docket` DataFrame to combine the comments with other metadata. There were 41 files with blanks for the comment that needed to be manually copied in.

This object is then serialized and saved to avoid this step going forward.

Note: for this code to work, you must have first run the web scraper (`scraper.py` - see README.md for instructions) to collect the comments - otherwise the `./Data/Comments/` directory is empty.

In [44]:
# Collect a list of all comment files (saved as .txt format)
comment_files = glob.glob('./Data/Comments/*.txt')
len(comment_files)

8187

In [63]:
# Create DF for each comment, concatenate into one DF
# Uncomment code to build the comment DF from the scraped files

# dfs = []

# for cf in comment_files:
#     try:
#         tmp = pd.read_csv(cf, sep=r'\n', header=None)
#         tmp.index = ['Document ID', 'Comment']
#         tmp = tmp.transpose()
#         dfs.append(tmp)
#     except ValueError as e:
#         print(cf)
#         continue


# com_df = pd.concat(dfs, ignore_index=True)

# com_df.head()

In [69]:
# Serialize com_df to pickled object
# com_df.to_pickle('./Data/comments_df.pkl')

In [70]:
# Read serialized DF vs re-building it
com_df = pd.read_pickle('./Data/comments_df.pkl')

com_df.head()

,Document ID,Comment
0,EPA-HQ-OW-2018-0149-8815,EPAs proposal would result in a profound weake...
1,EPA-HQ-OW-2018-0149-3940,I am commenting to urge that this new regulati...
2,EPA-HQ-OW-2018-0149-3798,It is absolutely laughable to pretend in regul...
3,EPA-HQ-OW-2018-0149-6186,I strongly oppose the EPAs proposed replacemen...
4,EPA-HQ-OW-2018-0149-7298,I write to express my opposition to the Enviro...


In [71]:
com_df.shape

(8187, 2)

In [73]:
# Merge docket DF with comments DF
# Uncomment code to perform the merge directly
# full_df = pd.merge(left=docket, right=com_df, how='right', on='Document ID')

# full_df.head()

,Document Title,Document Type,Attachment Count,Document ID,Posted Date,Received Date,Document SubType,Abstract,Status,Post Mark Date,File Type,Number Of Pages,Document Detail,Comment
0,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-6689,2019-04-29,2019-04-11,Public Comment,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...,I write to express my opposition to the Enviro...
1,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-6688,2019-04-29,2019-04-11,Public Comment,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...,I write to express my opposition to the Enviro...
2,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-7377,2019-05-01,2019-04-11,Public Comment,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...,Please abandon this proposed rollback of the 2...
3,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-6707,2019-04-29,2019-04-11,Public Comment,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...,"My husband and I have two wonderful daughters,..."
4,Comment submitted by T. Dahlin,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-7983,2019-05-01,2019-04-12,Public Comment,NaN,Posted,04/12/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...,Please do not change the definitions within th...


In [74]:
# Serialize full DF as pickle object
# full_df.to_pickle('./Data/full_df.pkl')

In [75]:
# Read serialized full DF vs re-building it
full_df = pd.read_pickle('./Data/full_df.pkl')

full_df.head()

,Document Title,Document Type,Attachment Count,Document ID,Posted Date,Received Date,Document SubType,Abstract,Status,Post Mark Date,File Type,Number Of Pages,Document Detail,Comment
0,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-6689,2019-04-29,2019-04-11,Public Comment,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...,I write to express my opposition to the Enviro...
1,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-6688,2019-04-29,2019-04-11,Public Comment,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...,I write to express my opposition to the Enviro...
2,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-7377,2019-05-01,2019-04-11,Public Comment,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...,Please abandon this proposed rollback of the 2...
3,Anonymous public comment,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-6707,2019-04-29,2019-04-11,Public Comment,NaN,Posted,04/11/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...,"My husband and I have two wonderful daughters,..."
4,Comment submitted by T. Dahlin,PUBLIC SUBMISSIONS,NaN,EPA-HQ-OW-2018-0149-7983,2019-05-01,2019-04-12,Public Comment,NaN,Posted,04/12/2019,NaN,1.0,https://www.regulations.gov/document?D=EPA-HQ-...,Please do not change the definitions within th...


In [76]:
full_df.shape

(8187, 14)

In [78]:
# Remove mostly and all-null columns
full_df.drop(['Attachment Count', 'Abstract', 'File Type'], axis=1, inplace=True)

In [79]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8187 entries, 0 to 8186
Data columns (total 11 columns):
Document Title      8187 non-null object
Document Type       8187 non-null object
Document ID         8187 non-null object
Posted Date         8187 non-null datetime64[ns]
Received Date       8187 non-null datetime64[ns]
Document SubType    8187 non-null object
Status              8187 non-null object
Post Mark Date      8134 non-null object
Number Of Pages     8187 non-null float64
Document Detail     8187 non-null object
Comment             8187 non-null object
dtypes: datetime64[ns](2), float64(1), object(8)
memory usage: 767.5+ KB
